In [1]:
# 구글 코랩 한글
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 9,599 kB of archives.
After this operation, 29.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 fonts-nanum all 20180306-3 [9,599 kB]
Fetched 9,599 kB in 1s (8,139 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 128276 files and di

In [2]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/colab/dataanalysis

Mounted at /content/drive
/content/drive/MyDrive/colab/dataanalysis


In [3]:
import numpy as np
import pandas as pd
import matplotlib as mp
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import missingno as msno
import folium

장소(location) 데이터 가져오기   

In [21]:
# delimiter에서 :: 을 사용해서 필드를 구분함
# 한글 사용해서 encoding은 euc-kr을 사용함
location =pd.read_csv('./pro1/location.dat', delimiter='::', encoding='euc-kr', engine='python', header=None)

In [23]:
# location 개수 확인. 20개의 데이터, 20 * 3 행렬
print(location.shape)

(20, 3)


location 데이터 확인 (헤더, 테일)   

In [24]:
location.head()

,0,1,2
0,1,남산서울타워,ETC|kids|friends
1,2,63스퀘어,ETC|kids|parents
2,3,광진교 8번가,ETC|alone|friends
3,4,녹사평 육교,ETC|alone|friends
4,21,창경궁,History|kids|parents|friends


In [25]:
location.tail()

,0,1,2
15,64,롯데월드 어드벤처,Attraction|kids|friends|alone
16,80,겸재정선미술관,Exhibition|alone|friends|parents
17,81,경찰박물관,Exhibition|kids|friends
18,83,국립중앙박물관,Exhibition|alone|friends|parents
19,84,국립항공박물관,Exhibition|alone|friends|kids


In [26]:
# location info 확인
location.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       20 non-null     int64 
 1   1       20 non-null     object
 2   2       20 non-null     object
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes


사용자 데이터 가져오기   

In [27]:
users = pd.read_csv('./pro1/users.dat', sep='::', engine='python', header=None)

In [29]:
# 유저 아이디 :성별 :나이
users.head()

,0,1,2
0,1,F,1
1,2,F,1
2,3,F,1
3,4,F,18
4,5,F,18


In [30]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       50 non-null     int64 
 1   1       50 non-null     object
 2   2       50 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.3+ KB


In [31]:
ratings = pd.read_csv('./pro1/ratings.dat',sep='::', engine='python',header=None)

In [32]:
#유저 아이디:장소 아이디:별점
ratings.head()

,0,1,2
0,1,1,4
1,1,2,4
2,1,41,5
3,1,81,5
4,2,21,4


In [33]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   0       200 non-null    int64
 1   1       200 non-null    int64
 2   2       200 non-null    int64
dtypes: int64(3)
memory usage: 4.8 KB


데이터 전처리    

In [34]:
print(dir(location))

['T', '_AXIS_LEN', '_AXIS_ORDERS', '_AXIS_TO_AXIS_NUMBER', '_HANDLED_TYPES', '__abs__', '__add__', '__and__', '__annotations__', '__array__', '__array_priority__', '__array_ufunc__', '__array_wrap__', '__bool__', '__class__', '__contains__', '__copy__', '__deepcopy__', '__delattr__', '__delitem__', '__dict__', '__dir__', '__divmod__', '__doc__', '__eq__', '__finalize__', '__floordiv__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__iand__', '__ifloordiv__', '__imod__', '__imul__', '__init__', '__init_subclass__', '__invert__', '__ior__', '__ipow__', '__isub__', '__iter__', '__itruediv__', '__ixor__', '__le__', '__len__', '__lt__', '__matmul__', '__mod__', '__module__', '__mul__', '__ne__', '__neg__', '__new__', '__nonzero__', '__or__', '__pos__', '__pow__', '__radd__', '__rand__', '__rdivmod__', '__reduce__', '__reduce_ex__', '__repr__', '__rfloordiv__', '__rmatmul__', '__rmod__', '__rmul__', '__ror__', '_

location 파일 column 이름 변경  

In [35]:
location.columns

Int64Index([0, 1, 2], dtype='int64')

In [36]:
location.rename(columns={0:'LocationID', 1:'LocationName', 2:'Theme'}, inplace=True)

In [37]:
# 데이터 확인
location.head()

,LocationID,LocationName,Theme
0,1,남산서울타워,ETC|kids|friends
1,2,63스퀘어,ETC|kids|parents
2,3,광진교 8번가,ETC|alone|friends
3,4,녹사평 육교,ETC|alone|friends
4,21,창경궁,History|kids|parents|friends


In [39]:
location.tail()

,LocationID,LocationName,Theme
15,64,롯데월드 어드벤처,Attraction|kids|friends|alone
16,80,겸재정선미술관,Exhibition|alone|friends|parents
17,81,경찰박물관,Exhibition|kids|friends
18,83,국립중앙박물관,Exhibition|alone|friends|parents
19,84,국립항공박물관,Exhibition|alone|friends|kids


In [40]:
# users도 이름 변경
users.rename(columns={0:'UserID', 1:'Gender', 2:'Age'}, inplace=True)

In [41]:
# 데이터 확인
users.head()

,UserID,Gender,Age
0,1,F,1
1,2,F,1
2,3,F,1
3,4,F,18
4,5,F,18


In [42]:
users.tail()

,UserID,Gender,Age
45,46,M,45
46,47,M,50
47,48,M,50
48,49,M,56
49,50,M,56


In [43]:
# ratings도 이름 변경하기
ratings.rename(columns={0:'UserID', 1:'LocationID', 2:'Rating'}, inplace=True)

In [44]:
# 데이터 확인 
ratings.head()

,UserID,LocationID,Rating
0,1,1,4
1,1,2,4
2,1,41,5
3,1,81,5
4,2,21,4


In [45]:
ratings.tail()

,UserID,LocationID,Rating
195,49,84,3
196,50,1,5
197,50,42,5
198,50,63,5
199,50,84,4


merge 기능을 사용해서 데이터 합치기   

In [46]:
data = ratings.merge(users).merge(location)

In [47]:
# 데이터 확인하기
data.head()

,UserID,LocationID,Rating,Gender,Age,LocationName,Theme
0,1,1,4,F,1,남산서울타워,ETC|kids|friends
1,4,1,5,F,18,남산서울타워,ETC|kids|friends
2,8,1,5,F,25,남산서울타워,ETC|kids|friends
3,10,1,4,F,25,남산서울타워,ETC|kids|friends
4,15,1,5,F,35,남산서울타워,ETC|kids|friends


In [48]:
#fancy indexing해서 필요한 컬럼만 가져오기.
recommendation_data = data[['UserID', 'LocationID','Rating']].copy()

In [49]:
recommendation_data.shape

(198, 3)

In [50]:
recommendation_data.head()

,UserID,LocationID,Rating
0,1,1,4
1,4,1,5
2,8,1,5
3,10,1,4
4,15,1,5


In [53]:
print(recommendation_data)

     UserID  LocationID  Rating
0         1           1       4
1         4           1       5
2         8           1       5
3        10           1       4
4        15           1       5
..      ...         ...     ...
193      26          22       5
194      31          22       5
195      33          22       5
196      37          22       5
197      41          22       5

[198 rows x 3 columns]


In [54]:
# 피봇 차트를 만들기 위해서 중복이 없어야 한다는데
# 일단 중복값 없애고 다른 변수에 넣어서 진행한다.
df = recommendation_data.drop_duplicates()

In [55]:
print(df)

     UserID  LocationID  Rating
0         1           1       4
1         4           1       5
2         8           1       5
3        10           1       4
4        15           1       5
..      ...         ...     ...
193      26          22       5
194      31          22       5
195      33          22       5
196      37          22       5
197      41          22       5

[197 rows x 3 columns]


피봇(pivot) 차트 만들기   

In [56]:
recommendation_pivot = df.pivot(index='UserID', columns='LocationID', values='Rating')

In [57]:
recommendation_pivot

LocationID,1,2,3,4,21,22,23,24,41,42,43,44,61,62,63,64,80,81,83,84
UserID,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN
2,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,4.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,5.0,NaN,5.0,NaN,NaN,NaN,4.0,NaN
4,5.0,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,5.0,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,5.0,5.0,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,5.0,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN
9,NaN,NaN,4.0,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,NaN,NaN,NaN,NaN,NaN


In [58]:
# Nan 데이터를 0으로 바꾸기
recommendation_pivot.fillna(0,inplace=True)

In [59]:
# 데이터 확인
recommendation_pivot

LocationID,1,2,3,4,21,22,23,24,41,42,43,44,61,62,63,64,80,81,83,84
UserID,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0
2,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,5.0,0.0,5.0,0.0,0.0,0.0,4.0,0.0
4,5.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,4.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,5.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,5.0,5.0,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,5.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0
9,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0


In [60]:
data.head()

,UserID,LocationID,Rating,Gender,Age,LocationName,Theme
0,1,1,4,F,1,남산서울타워,ETC|kids|friends
1,4,1,5,F,18,남산서울타워,ETC|kids|friends
2,8,1,5,F,25,남산서울타워,ETC|kids|friends
3,10,1,4,F,25,남산서울타워,ETC|kids|friends
4,15,1,5,F,35,남산서울타워,ETC|kids|friends


In [61]:
data.tail()

,UserID,LocationID,Rating,Gender,Age,LocationName,Theme
193,26,22,5,M,1,덕수궁,History|kids|parents|friends
194,31,22,5,M,18,덕수궁,History|kids|parents|friends
195,33,22,5,M,18,덕수궁,History|kids|parents|friends
196,37,22,5,M,25,덕수궁,History|kids|parents|friends
197,41,22,5,M,35,덕수궁,History|kids|parents|friends


In [ ]:
# 데이터 전처리 과정 끝!

In [65]:
# userID가 1인 사람이 간 장소 목록
loc_user_1=recommendation_pivot.loc[1]
loc_user_1[recommendation_pivot.loc[1]>0]

LocationID
1     4.0
2     4.0
41    5.0
81    5.0
Name: 1, dtype: float64

In [66]:
# UserID가 1인 사람이 매긴 평점별 장소 개수
recommendation_pivot.loc[1][recommendation_pivot.loc[1]>0].value_counts()

4.0    2
5.0    2
Name: 1, dtype: int64

In [67]:
recommendation_pivot.loc[1][recommendation_pivot.loc[1]>0].unique()

array([4., 5.])

In [68]:
# 함수를 정의해서 일반화 시키다.
def location_been(user_id):
  return recommendation_pivot.loc[user_id][recommendation_pivot.loc[user_id]>0]

In [69]:
# 2번 유저가 가본 장소
location_been(2)

LocationID
21    4.0
43    4.0
64    5.0
84    4.0
Name: 2, dtype: float64

상관분석. corr 이용하기   

In [70]:
users[['UserID','Age']].corr()

,UserID,Age
UserID,1.000000,0.440978
Age,0.440978,1.000000


In [71]:
recommendation_pivot.head(1)

LocationID,1,2,3,4,21,22,23,24,41,42,43,44,61,62,63,64,80,81,83,84
UserID,,,,,,,,,,,,,,,,,,,,
1,4.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0


In [74]:
# 전치행렬로 변환
small_test = recommendation_pivot.T.iloc[:100,:100]
small_test.head(1)

UserID,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
LocationID,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,0.0,5.0,0.0,0.0,0.0,5.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0


In [75]:
# 상관관계 파악
small_test_corr = small_test.corr()
small_test_corr

UserID,1,2,3,4,5,6,7,8,9,10,...,41,42,43,44,45,46,47,48,49,50
UserID,,,,,,,,,,,,,,,,,,,,,
1,1.000000,-0.246499,-0.246201,0.074461,-0.246814,-0.248093,-0.248093,0.691369,-0.246499,0.680851,...,0.330529,0.391315,-0.246201,-0.248093,-0.015877,0.133739,-0.248093,0.009956,0.009956,0.041857
2,-0.246499,1.000000,-0.246499,-0.242687,0.135415,0.116891,0.043834,-0.247113,-0.246798,0.156277,...,-0.247113,-0.242687,0.075722,-0.248394,0.058909,0.075722,0.336062,0.024626,0.323657,-0.002295
3,-0.246201,-0.246499,1.000000,-0.242394,-0.246814,-0.248093,-0.248093,-0.246814,0.156277,-0.246201,...,-0.246814,0.153674,0.057751,0.372140,0.041857,0.057751,-0.248093,-0.238935,0.175883,0.402697
4,0.074461,-0.242687,-0.242394,1.000000,-0.242998,-0.244257,-0.244257,0.133160,0.294752,0.074461,...,0.057928,0.091660,0.312102,0.114944,-0.242998,0.074461,0.043104,0.024216,-0.235241,0.133160
5,-0.246814,0.135415,-0.246814,-0.242998,1.000000,0.733045,0.078540,0.026731,-0.002295,0.114025,...,-0.247430,-0.242998,0.041857,0.013090,0.095271,-0.246814,0.078540,-0.003152,-0.239531,-0.247430
6,-0.248093,0.116891,-0.248093,-0.244257,0.733045,1.000000,0.375000,0.013090,-0.248394,0.096481,...,-0.248712,-0.244257,0.027566,-0.250000,0.078540,-0.248093,0.062500,0.361158,-0.240772,-0.248712
7,-0.248093,0.043834,-0.248093,-0.244257,0.078540,0.375000,1.000000,-0.248712,-0.248394,-0.248093,...,0.078540,-0.244257,-0.248093,-0.250000,-0.248712,-0.248093,0.375000,0.135434,-0.240772,-0.248712
8,0.691369,-0.247113,-0.246814,0.133160,0.026731,0.013090,-0.248712,1.000000,-0.247113,0.402697,...,0.095271,-0.017303,-0.246814,-0.248712,0.300891,-0.246814,-0.248712,-0.003152,-0.003152,0.095271
9,-0.246499,-0.246798,0.156277,0.294752,-0.002295,-0.248394,-0.248394,-0.247113,1.000000,-0.246499,...,-0.247113,0.026032,0.333499,0.336062,-0.247113,0.333499,-0.248394,0.112576,-0.239225,0.135415


In [78]:
# userID 1인 사람과 상관관계가 높은 상위 3명 뽑기
small_test_corr[5].sort_values(ascending=False)[1:4]

UserID
39    0.752191
6     0.733045
30    0.712132
Name: 5, dtype: float64

In [79]:
# 함수 정의를 통한 일반화
def nearest_user(user_id,n):
  return small_test_corr.loc[user_id].sort_values(ascending=False)[1:n+1]

In [82]:
# 5번 유저와 상관관계가 높은 3명 뽑기
nearest_user(5,3)

UserID
39    0.752191
6     0.733045
30    0.712132
Name: 5, dtype: float64

상관관계가 가장 높은 39번과 5번 유저가 간 장소를 비교

In [83]:
# 39번이 간 장소를 출력
location_been(39)

LocationID
4     5.0
24    5.0
64    5.0
81    3.0
Name: 39, dtype: float64

In [84]:
# 5번이 간 장소를 출력
location_been(5)

LocationID
4     4.0
24    5.0
64    5.0
80    5.0
Name: 5, dtype: float64

39번은 가고 5번은 가지 않은 장소를 추천한다.   

In [103]:
unbeen = set(location_been(39).index) - set(location_been(5).index)
len(unbeen)
unbeen

# 39번은 갔지만 5번 유저가 가지 않은 장소의 개수 = 1

{81}

In [116]:
# 안가본 장소 출력
unbeen_location = recommendation_data[recommendation_data.LocationID.isin(unbeen)]
unbeen_location
# location_been(39)

,UserID,LocationID,Rating
33,1,81,5
34,8,81,5
35,10,81,5
36,15,81,4
37,19,81,3
38,21,81,3
39,26,81,5
40,27,81,5
41,28,81,5
42,28,81,5


In [122]:
# 39번 유저가 안 가본 장소 중에 평점이 5인 장소..?
# 왜 데이터가 한 개도 안나오는거지 -> 데이터가 부족해서!
unbeen_location[(unbeen_location.Rating==3)&(unbeen_location.UserID==39)]

,UserID,LocationID,Rating
45,39,81,3


In [123]:
# 추천할 여행지를 가지고 온다.
# 데이터가 부족해서 안가지고 오는 듯 -> 평점을 3으로 하니까 하나 나오기는 하네
unbeen_location[(unbeen_location.Rating==3)&(unbeen_location.UserID==39)].merge(location)

,UserID,LocationID,Rating,LocationName,Theme
0,39,81,3,경찰박물관,Exhibition|kids|friends


In [127]:
# 함수 정의로 일반화
def unbeen_loction(user_id, other_id):
  unbeen = set(location_been(user_id).index)-set(location_been(other_id).index)
  unbeen_location = recommendation_data[recommendation_data.LocationID.isin(unbeen)]
  return unbeen_location[(unbeen_location.Rating==3)&(unbeen_location.UserID==user_id)].merge(location)

In [128]:
# 39번은 보고 5번은 가지 않은 장소 중에서 평점 3인 장소를 유저 5번에 추천
unbeen_loction(39,5)

,UserID,LocationID,Rating,LocationName,Theme
0,39,81,3,경찰박물관,Exhibition|kids|friends


테스트.. 7번과 유사한 2명을 뽑아서 한명이라도 평점이 5점을 준 장소를 추천하기   